# 1 Introduction

### 1.1 Createn Agent

#### Setup Ollama Dependencis
- Look at note on how to start ollama in the background
- window 1: ollama serve
- window 2: ollama run llama3.2

In [1]:
# Set the OLLAMA_BASE_URL environment variable
import os
ollama_url = 'http://127.0.0.1:11434' # from running ollama serve
os.environ["OLLAMA_BASE_URL"] = ollama_url

# Verify that it has been set
print(f"OLLAMA_BASE_URL is set to: {os.getenv('OLLAMA_BASE_URL')}")

OLLAMA_BASE_URL is set to: http://127.0.0.1:11434


#### Clean up previous agents

In [11]:
from letta import ChatMemory, EmbeddingConfig, LLMConfig, create_client
client = create_client()

for agent_state in client.list_agents():
    print(f'AgentId: {agent_state.id}')
    client.delete_agent(agent_id=agent_state.id)


AgentId: agent-eb689b04-26e6-43bb-b672-d023fdbf8329


#### Create Client

In [12]:
from letta import ChatMemory, EmbeddingConfig, LLMConfig, create_client
from letta.prompts import gpt_system

client = create_client()

llm_config = LLMConfig(
    # llama model versions: https://github.com/ollama/ollama
    model="llama3.2:1b", #"llama3.2", #ollama run llama3.2:1b
    model_endpoint_type="ollama", # llamacp
    model_endpoint=ollama_url,
    context_window=16384
)
embedding_config = EmbeddingConfig.default_config(model_name="text-embedding-ada-002")

# create a new agent
agent_state = client.create_agent(
    # agent's name (unique per-user, autogenerated if not provided)
    name="agent_name",
    # in-context memory representation with human/persona blocks
    memory=ChatMemory(
      human="Name: Sarah", 
      persona="You are a helpful assistant that loves emojis"
    ),
    # LLM model & endpoint configuration
    llm_config=llm_config,
    # embedding model & endpoint configuration (cannot be changed)
    embedding_config=embedding_config,
    # system instructions for the agent (defaults to `memgpt_chat`)
    system=gpt_system.get_system_text("memgpt_chat"),
    # whether to include base letta tools (default: True)
    include_base_tools=True,
    # list of additional tools (by name) to add to the agent
    tools=[],
)
print(f"Created agent with name {agent_state.name} and unique ID {agent_state.id}")

Created agent with name agent_name and unique ID agent-276387cc-599b-4cbb-8652-0cab84ab38f1


In [ ]:
# get the agent by ID
agent_state = client.get_agent(agent_id=agent_state.id)

# get the agent by name
agent_id = client.get_agent_id(agent_name=agent_state.name)
agent_state = client.get_agent(agent_id=agent_id)

print(f'agent_state={agent_state}')

### 1.2 Send a message Client

In [13]:
print(1)
# Message an agent
response = client.send_message(
  agent_id=agent_state.id, 
  role="user", 
  message="hello There.  I like pickleball alot.  Tel me some paddles I can get."
)
print(2)
print("Usage", response.usage)

print(3)
print("Agent messages", response.messages)

1
Archival insert error Connection error.
2
Usage completion_tokens=1239 prompt_tokens=66620 total_tokens=67574 step_count=17
3
Agent messages [InternalMonologue(id='message-e66dfc12-e603-4825-ad96-173068da32e4', date=datetime.datetime(2024, 11, 21, 0, 26, 51, 486407, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Bootup sequence complete. Persona activated. Testing messaging functionality.'), FunctionCallMessage(id='message-e66dfc12-e603-4825-ad96-173068da32e4', date=datetime.datetime(2024, 11, 21, 0, 26, 51, 486407, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='conversation_search', arguments='{\n  "query": "pickleball paddles",\n  "page": "0",\n  "request_heartbeat": true\n}', function_call_id='f3c0ab92-6fab-4a69-96f9-36568')), FunctionReturn(id='message-1c64e8b6-4872-4d8e-9e27-55fd37e34bf1', date=datetime.datetime(2024, 11, 21, 0, 26, 51, 505014, tzinfo=datetime.timezone.utc), message_type='funct

In [ ]:
client.list_agents()

#client.delete_agent(agent_id=agent_state.id)

# 2 Tools
- Agents map NLU to actions --> tools
- Documentation: https://docs.letta.com/agents/tools
- Letta comes with a default set of tools: **send_message, conversation_search, conversation_search_date, archival_memory_insert, archival_memory_search**

## 2.1 Define My Tool

In [21]:
def roll_d20(self) -> str:
    """
    Simulate the roll of a 20-sided die (d20).

    This function generates a random integer between 1 and 20, inclusive,
    which represents the outcome of a single roll of a d20.

    Returns:
        int: A random integer between 1 and 20, representing the die roll.

    Example:
        >>> roll_d20()
        15  # This is an example output and may vary each time the function is called.
    """
    import random

    dice_role_outcome = random.randint(1, 20)
    output_string = f"You rolled a {dice_role_outcome}"
    return output_string


from letta import create_client
if not client:
    client = create_client()

tool = client.create_tool(roll_d20)    
tool


Tool(id='tool-ab4e96ec-e51d-4120-8c2a-3ea2b30a9f1c', description=None, source_type='python', module=None, organization_id='org-00000000-0000-4000-8000-000000000000', name='roll_d20', tags=[], source_code='def roll_d20(self) -> str:\n    """\n    Simulate the roll of a 20-sided die (d20).\n\n    This function generates a random integer between 1 and 20, inclusive,\n    which represents the outcome of a single roll of a d20.\n\n    Returns:\n        int: A random integer between 1 and 20, representing the die roll.\n\n    Example:\n        >>> roll_d20()\n        15  # This is an example output and may vary each time the function is called.\n    """\n    import random\n\n    dice_role_outcome = random.randint(1, 20)\n    output_string = f"You rolled a {dice_role_outcome}"\n    return output_string\n', json_schema={'name': 'roll_d20', 'description': 'Simulate the roll of a 20-sided die (d20).', 'parameters': {'type': 'object', 'properties': {'request_heartbeat': {'type': 'boolean', 'descr

## 2.2 Create Agent with Tool

#### Delete Previous Agents

In [29]:
client = create_client()

for agent_state in client.list_agents():
    print(f'Deleting AgentId: {agent_state.id}')
    client.delete_agent(agent_id=agent_state.id)


Deleting AgentId: agent-ad8d46c2-d21e-4fa1-8176-76ed41f03bce


#### Create Agent With My Tool
- Tool Rules: https://docs.letta.com/agents/tools
    * Defines behaviors of tools, ie when it is called and what happens after it's called 

In [32]:
from letta.schemas.tool_rule import TerminalToolRule
from letta import EmbeddingConfig, LLMConfig

llm_config = LLMConfig(
    # llama model versions: https://github.com/ollama/ollama
    model="llama3.2:1b", #"llama3.2", #ollama run llama3.2:1b
    model_endpoint_type="ollama", # llamacp
    model_endpoint=ollama_url,
    context_window=16384
)
embedding_config = EmbeddingConfig.default_config(model_name="text-embedding-ada-002")

# create a new agent
agent_state = client.create_agent(
    # agent's name (unique per-user, autogenerated if not provided)
    #name="agent_name",
    
    # in-context memory representation with human/persona blocks
    #memory=ChatMemory(
    #  human="Name: Sarah", 
    #  persona="You are a helpful assistant that loves emojis"
    #),
    
    # LLM model & endpoint configuration
    llm_config=llm_config,
    
    # embedding model & endpoint configuration (cannot be changed)
    embedding_config=embedding_config,
    
    # system instructions for the agent (defaults to `memgpt_chat`)
    #system=gpt_system.get_system_text("memgpt_chat"),
    
    # whether to include base letta tools (default: True)
    # include_base_tools=True,

    # list of additional tools (by name) to add to the agent
    tools=[tool.name],

    # add tool rules that terminate execution after specific tools
    tool_rules=[
        # exit after roll_d20 is called
        TerminalToolRule(tool_name=tool.name),
        # exit after send_message is called (default behavior)
        TerminalToolRule(tool_name="send_message"),
    ],
)
print(f"Created agent with name {agent_state.name} with tools {agent_state.tools}")


Created agent with name SereneXylophone with tools ['roll_d20', 'send_message', 'conversation_search', 'conversation_search_date', 'archival_memory_insert', 'archival_memory_search', 'core_memory_append', 'core_memory_replace']


#### Send a message to trigger my tool

In [37]:
# Message an agent
response = client.send_message(agent_id=agent_state.id, role="user", message="Roll the dice")
print("Usage", response.usage)

print("")
print("Agent messages", response.messages)

Letta.letta.server.server - ERROR - Error in server._step: Summarize error: tried to run summarize, but couldn't find enough messages to compress [len=1 <= 1]


Traceback (most recent call last):
  File "/Users/chang/Documents/dev/git/project/walle/venv/lib/python3.11/site-packages/letta/agent.py", line 950, in inner_step
    response = self._get_ai_reply(
               ^^^^^^^^^^^^^^^^^^^
  File "/Users/chang/Documents/dev/git/project/walle/venv/lib/python3.11/site-packages/letta/agent.py", line 568, in _get_ai_reply
    raise e
  File "/Users/chang/Documents/dev/git/project/walle/venv/lib/python3.11/site-packages/letta/agent.py", line 531, in _get_ai_reply
    response = create(
               ^^^^^^^
  File "/Users/chang/Documents/dev/git/project/walle/venv/lib/python3.11/site-packages/letta/llm_api/llm_api_tools.py", line 97, in wrapper
    raise e
  File "/Users/chang/Documents/dev/git/project/walle/venv/lib/python3.11/site-packages/letta/llm_api/llm_api_tools.py", line 66, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/chang/Documents/dev/git/project/walle/venv/lib/python3.11/site-packages/le

None


LLMError: Summarize error: tried to run summarize, but couldn't find enough messages to compress [len=1 <= 1]